In [1]:
!pip3 install tweepy python-twitter kafka-python python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import tweepy
import json
from tweepy import OAuthHandler
from tweepy import Stream
from kafka import KafkaConsumer, KafkaProducer

In [3]:
from dotenv import load_dotenv

load_dotenv()

CONSUMER_KEY = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET = os.getenv('CONSUMER_SECRET')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
ACCESS_SECRET = os.getenv('ACCESS_TOKEN_SECRET')
BEARER_TOKEN = os.getenv('BEARER_TOKEN')

In [4]:
topic_name = 'twitter_data'
search_input = "Lula, Bolsonaro, Tebet, Ciro Gomes, #Bolsonaro2022, #Lula2022"

kafka_producer = KafkaProducer(bootstrap_servers='localhost:9092')

client = tweepy.Client(bearer_token=BEARER_TOKEN, 
                       consumer_key=CONSUMER_KEY, 
                       consumer_secret=CONSUMER_SECRET, 
                       access_token=ACCESS_TOKEN, 
                       access_token_secret=ACCESS_SECRET)

query = 'Lula -is:retweet'

tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'],
                                     user_fields=['profile_image_url'], 
                                     expansions=["author_id", 
                                                 "entities.mentions.username", 
                                                 "geo.place_id", 
                                                 "in_reply_to_user_id", 
                                                 "referenced_tweets.id", 
                                                 "referenced_tweets.id.author_id"],
                                     max_results=10)

for tweet in tweets.data:
    print(tweet.data)
    kafka_producer.send(topic_name, json.dumps(tweet.data).encode('utf-8')).get(timeout=10)

{'entities': {'mentions': [{'start': 17, 'end': 30, 'username': 'JovemPanNews', 'id': '106175018'}]}, 'id': '1570767876315033602', 'text': 'Lula  criminoso  @JovemPanNews #jornaldamanha #3em1 #ospingosnosis #morningshow  só para constar ✍🏽 https://t.co/Tp8id2A5he', 'author_id': '1243811048559345665', 'created_at': '2022-09-16T13:33:39.000Z', 'context_annotations': [{'domain': {'id': '10', 'name': 'Person', 'description': 'Named people in the world like Nelson Mandela'}, 'entity': {'id': '862070591737675776', 'name': 'Luiz Inácio "Lula" da Silva', 'description': 'Brazilian 35th president.'}}, {'domain': {'id': '35', 'name': 'Politician', 'description': 'Politicians in the world, like Joe Biden'}, 'entity': {'id': '862070591737675776', 'name': 'Luiz Inácio "Lula" da Silva', 'description': 'Brazilian 35th president.'}}, {'domain': {'id': '131', 'name': 'Unified Twitter Taxonomy', 'description': 'A taxonomy view into the Semantic Core knowledge graph'}, 'entity': {'id': '847878884917886977